In [1]:
from bpredict import *
import numpy as np

In [2]:
class Perceptron(object):
    """A basic implementation of a perceptron."""
    def __init__(self, length, threshold=1.0):
        self._weights = np.zeros(length + 1)
        self._weights[0] = 1.0
        self._threshold = threshold

    def predict(self, history):
        """Get a prediction based on the history."""
        hist = np.concatenate(([1], history))
        return np.dot(self._weights, hist) >= 0

    def update(self, history, taken):
        """Update the perceptron once the output of a branch is known.
        :param taken: -1 if the branch was not taken and 1 if it was taken.
        """
        hist = np.concatenate(([1], history))
        y = np.dot(self._weights, hist)
        if np.sign(y) != np.sign(taken) or abs(y) < self._threshold:
            self._weights += taken * hist

In [3]:
class PerceptronPredictor(BasePredictor):
    """A perceptron predictor that ignores speculative execution."""
    def __init__(self, nperceptrons, histlength, threshold=1.0):
        super(PerceptronPredictor, self).__init__()
        self._nperceptrons = nperceptrons
        self._histlength = histlength
        self._globalhistory = np.zeros(histlength)

        self._table = [Perceptron(histlength, threshold=threshold) for _ in range(nperceptrons)]

    def lookup(self, tid, branch_addr, bp_history):
        index = self._get_index(branch_addr)
        return self._table[index].predict(self._globalhistory)

    def update(self, tid, branch_addr, taken, bp_history, squashed):
        if squashed:
            return

        index = self._get_index(branch_addr)
        t = 1 if taken else -1
        self._table[index].update(self._globalhistory, t)
        
        self._globalhistory = np.roll(self._globalhistory, -1)
        self._globalhistory[-1] = t

    def _get_index(self, branch_addr):
        return ((branch_addr // 4) % len(self._table))

In [9]:
benchmark = '../benchmarks/picosat/picosat'
args = ('../benchmarks/picosat/inputs/uuf50-01.cnf', )

#benchmark = '../benchmarks/blackscholes/blackscholes'
#args = (1, '../benchmarks/blackscholes/inputs/tiny.input', '/dev/null')

#benchmark = '../benchmarks/sha256sum/sha256sum.alpha'
#args = ('../benchmarks/sha256sum/small.bin', )

#benchmark = '../benchmarks/streamcluster/streamcluster'
#args = (10, 20, 3, 100, 100, 100, '/dev/zero', '/dev/null', 1)

In [11]:
perc_pred = PerceptronPredictor(nperceptrons=256, histlength=32, threshold=48)

for _ in range(4):
    perc_runner = ExternalRunner(perc_pred, benchmark, args=args)
    perc_runner.run()

    predicted = perc_runner.stats[0].find('condPredicted')[0].values[0]
    incorrect = perc_runner.stats[0].find('condIncorrect')[0].values[0]

    print('Misprediction rate: %f' % (incorrect / predicted))

Misprediction rate: 0.042518
Misprediction rate: 0.039980
Misprediction rate: 0.040051
Misprediction rate: 0.039773


In [12]:
local_pred = Local2BitPredictor(8192)

local_runner = ExternalRunner(local_pred, benchmark, args=args)
local_runner.run()

predicted = local_runner.stats[0].find('condPredicted')[0].values[0]
incorrect = local_runner.stats[0].find('condIncorrect')[0].values[0]

print('Misprediction rate: %f' % (incorrect / predicted))

Misprediction rate: 0.066731
